In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [1]:
from datasets import load_dataset
val_data=load_dataset("wmt16","de-en", split="validation")
test_data=load_dataset("wmt16","de-en", split="test")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for wmt16 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at wmt16/wmt16.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for wmt16 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at wmt16/wmt16.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
import torch
device = torch.device("mps")
print("Using device:", device)

Using device: mps


In [4]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small").to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
from tqdm import tqdm
def generate_translations(data):
    translations = []
    for example in tqdm(data, desc="Translating"):
        input_text = "translate English to German: " + example["translation"]["en"]
        inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
        outputs = model.generate(inputs.input_ids,max_length=500, attention_mask=inputs.attention_mask)
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translated_text)
    return translations





In [6]:
validation_translations = generate_translations(val_data)
test_translations = generate_translations(test_data)

Translating: 100%|██████████| 2999/2999 [41:58<00:00,  1.19it/s]  


In [9]:
import json

with open("validation_translations.json", "w") as f:
    json.dump(validation_translations, f)


with open("test_translations.json", "w") as f:
    json.dump(test_translations, f)


In [3]:
import pickle

with open("validation_translations.pkl", "wb") as f:
    pickle.dump(validation_translations, f)

# Save test translations using pickle
with open("test_translations.pkl", "wb") as f:
    pickle.dump(test_translations, f)


NameError: name 'test_translations' is not defined

In [3]:
import json
validation_translations=json.load(open("validation_translations.json"))
test_translations=json.load(open("test_translations.json"))

In [2]:
print(validation_translations[0])

In Tokio treffen sich die Premierminister Indiens und Japans


In [2]:
validation_refs = [example["translation"]["de"] for example in val_data]
test_refs = [example["translation"]["de"] for example in test_data]

'''
#calculate bleu score

import nltk
from nltk.translate.bleu_score import corpus_bleu
def calculate_bleu_scores(actual_sentences, translated_sentences):
    bleu_1 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(1, 0, 0, 0))
    bleu_2 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 1, 0, 0))
    bleu_3 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 1, 0))
    bleu_4 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 0, 1))
    return bleu_1, bleu_2, bleu_3, bleu_4




validation_bleu_1, validation_bleu_2, validation_bleu_3, validation_bleu_4 = calculate_bleu_scores(validation_refs, validation_translations)
test_bleu_1, test_bleu_2, test_bleu_3, test_bleu_4 = calculate_bleu_scores(test_refs, test_translations)


print("Validation BLEU-1 Score:", validation_bleu_1)
print("Validation BLEU-2 Score:", validation_bleu_2)
print("Validation BLEU-3 Score:", validation_bleu_3)
print("Validation BLEU-4 Score:", validation_bleu_4)

print("\nTest BLEU-1 Score:", test_bleu_1)
print("Test BLEU-2 Score:", test_bleu_2)
print("Test BLEU-3 Score:", test_bleu_3)
print("Test BLEU-4 Score:", test_bleu_4)

'''


'\n#calculate bleu score\n\nimport nltk\nfrom nltk.translate.bleu_score import corpus_bleu\ndef calculate_bleu_scores(actual_sentences, translated_sentences):\n    bleu_1 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(1, 0, 0, 0))\n    bleu_2 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 1, 0, 0))\n    bleu_3 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 1, 0))\n    bleu_4 = corpus_bleu([[ref.split()] for ref in actual_sentences], [hyp.split() for hyp in translated_sentences], weights=(0, 0, 0, 1))\n    return bleu_1, bleu_2, bleu_3, bleu_4\n\n\n\n\nvalidation_bleu_1, validation_bleu_2, validation_bleu_3, validation_bleu_4 = calculate_bleu_scores(validation_refs, validation_translations)\ntest_bleu_1, test_bleu_2, test_bleu_3, test_bleu_4 = calculate_bleu_scores(test_refs, t

In [6]:
import evaluate

bleu = evaluate.load("bleu")



In [7]:
import evaluate

bleu = evaluate.load("bleu")
results_1 = bleu.compute(predictions=validation_translations, references=validation_refs, max_order=1)
print(results_1)
results_2 = bleu.compute(predictions=validation_translations, references=validation_refs, max_order=2)
print(results_2)
results_3 = bleu.compute(predictions=validation_translations, references=validation_refs, max_order=3)
print(results_3)
results_4 = bleu.compute(predictions=validation_translations, references=validation_refs, max_order=4)
print(results_4)

{'bleu': 0.5896815800080613, 'precisions': [0.5896815800080613], 'brevity_penalty': 1.0, 'length_ratio': 1.0089923181201987, 'translation_length': 44658, 'reference_length': 44260}
{'bleu': 0.4406826405673354, 'precisions': [0.5896815800080613, 0.32933229777118783], 'brevity_penalty': 1.0, 'length_ratio': 1.0089923181201987, 'translation_length': 44658, 'reference_length': 44260}
{'bleu': 0.34410406607004834, 'precisions': [0.5896815800080613, 0.32933229777118783, 0.2098058180194926], 'brevity_penalty': 1.0, 'length_ratio': 1.0089923181201987, 'translation_length': 44658, 'reference_length': 44260}
{'bleu': 0.27436246022629457, 'precisions': [0.5896815800080613, 0.32933229777118783, 0.2098058180194926, 0.1390686826865124], 'brevity_penalty': 1.0, 'length_ratio': 1.0089923181201987, 'translation_length': 44658, 'reference_length': 44260}


In [8]:
import evaluate

bleu = evaluate.load("bleu")
results_1 = bleu.compute(predictions=test_translations, references=test_refs, max_order=1)
print(results_1)
results_2 = bleu.compute(predictions=test_translations, references=test_refs, max_order=2)
print(results_2)
results_3 = bleu.compute(predictions=test_translations, references=test_refs, max_order=3)
print(results_3)
results_4 = bleu.compute(predictions=test_translations, references=test_refs, max_order=4)
print(results_4)

{'bleu': 0.6173815483698936, 'precisions': [0.6225082455152442], 'brevity_penalty': 0.9917644510216771, 'length_ratio': 0.9917981777274251, 'translation_length': 62155, 'reference_length': 62669}
{'bleu': 0.4767904215770271, 'precisions': [0.6225082455152442, 0.3712725674487795], 'brevity_penalty': 0.9917644510216771, 'length_ratio': 0.9917981777274251, 'translation_length': 62155, 'reference_length': 62669}
{'bleu': 0.3812786762509227, 'precisions': [0.6225082455152442, 0.3712725674487795, 0.2458464661573802], 'brevity_penalty': 0.9917644510216771, 'length_ratio': 0.9917981777274251, 'translation_length': 62155, 'reference_length': 62669}
{'bleu': 0.31045530684044015, 'precisions': [0.6225082455152442, 0.3712725674487795, 0.2458464661573802, 0.16898971088915224], 'brevity_penalty': 0.9917644510216771, 'length_ratio': 0.9917981777274251, 'translation_length': 62155, 'reference_length': 62669}


In [19]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=test_translations, references=test_refs)
print(results)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chehakmalhotra/nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chehakmalhotra/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/chehakmalhotra/nltk_data...


{'meteor': 0.5772148087831891}


In [20]:
meteor = evaluate.load('meteor')
results = meteor.compute(predictions=validation_translations, references=validation_refs)
print(results)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chehakmalhotra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chehakmalhotra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/chehakmalhotra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.5458072768029277}


In [4]:
from evaluate import load
bertscore = load("bertscore")
results = bertscore.compute(predictions=validation_translations, references=validation_refs, model_type="distilbert-base-uncased")
print(results)

precision_scores = results['precision']
recall_scores = results['recall']
f1_scores = results['f1']

# Calculate the average
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1-score:", average_f1)

{'precision': [0.9398015737533569, 0.9134738445281982, 0.972328782081604, 0.9294741749763489, 0.9352404475212097, 0.9204177856445312, 0.9758858680725098, 0.9042976498603821, 0.9292457103729248, 0.9683436155319214, 0.9190176129341125, 0.9502357244491577, 0.9634466767311096, 0.8888418078422546, 0.9270459413528442, 0.8978112936019897, 0.963437020778656, 0.8796160817146301, 0.9129408001899719, 0.8957769274711609, 0.8975597620010376, 0.9332404732704163, 0.8886365294456482, 0.8973479866981506, 0.917441725730896, 0.9008811712265015, 0.9703555107116699, 0.9312429428100586, 0.9154028296470642, 0.9336246252059937, 0.9534447193145752, 0.9306413531303406, 0.9484187960624695, 0.9184016585350037, 0.9218538403511047, 0.9365274906158447, 0.9580791592597961, 0.9199740290641785, 0.8986025452613831, 0.9827415943145752, 0.9576029777526855, 0.9426041841506958, 0.9363282322883606, 0.9763807058334351, 0.9670500755310059, 0.9318403601646423, 0.9376487731933594, 0.9321423768997192, 0.918660044670105, 0.9163491

In [5]:

results = bertscore.compute(predictions=test_translations, references=test_refs, model_type="distilbert-base-uncased")
print(results)
precision_scores = results['precision']
recall_scores = results['recall']
f1_scores = results['f1']

# Calculate the average
average_precision = sum(precision_scores) / len(precision_scores)
average_recall = sum(recall_scores) / len(recall_scores)
average_f1 = sum(f1_scores) / len(f1_scores)

print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1-score:", average_f1)

{'precision': [0.8257314562797546, 0.9302700757980347, 0.9552820920944214, 0.9294861555099487, 0.9019855260848999, 0.8721365332603455, 0.9045155048370361, 0.9350815415382385, 0.9327290058135986, 0.8568301200866699, 0.9044193029403687, 0.9422681927680969, 0.909488320350647, 0.9423784017562866, 0.6688590049743652, 0.9031323194503784, 0.937346339225769, 0.9334549903869629, 0.9349777698516846, 0.8980790972709656, 0.9710081219673157, 0.9186726212501526, 0.938202977180481, 0.9535342454910278, 0.9370960593223572, 0.9554904699325562, 0.9096150398254395, 0.9946751594543457, 0.9056703448295593, 0.9384157657623291, 0.9999999403953552, 0.9375860691070557, 0.8972858190536499, 0.9828983545303345, 0.9531012177467346, 0.9641745686531067, 0.9395007491111755, 0.8602300882339478, 0.9444053173065186, 0.9556854963302612, 0.956149697303772, 0.855496883392334, 0.9358809590339661, 0.9147244691848755, 0.9366319179534912, 0.9408829808235168, 0.9692210555076599, 0.8468997478485107, 0.9403102993965149, 0.99999994